In [109]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

df = pd.read_csv('results.csv')
df = df[df['trial name'] == 'house-grey-box_blur-10iter']
df

,trial name,items,id,type,threads,completion time,service time,bandwidth,speedup,scalability,efficiency
0,house-grey-box_blur-10iter,84,1,seq,1,1652140,1,1.0,1.00,1.00,1.00
1,house-grey-box_blur-10iter,84,1,threads,1,1363831,16236,0.0,1.21,1.01,1.21
2,house-grey-box_blur-10iter,84,1,threads,2,780159,9287,0.0,2.12,1.76,1.06
3,house-grey-box_blur-10iter,84,1,threads,4,402076,4786,0.0,4.11,3.42,1.03
4,house-grey-box_blur-10iter,84,1,threads,6,381370,4540,0.0,4.33,3.61,0.72
...,...,...,...,...,...,...,...,...,...,...,...
170,house-grey-box_blur-10iter,84,5,ff,24,390426,4647,0.0,3.81,3.90,0.16
171,house-grey-box_blur-10iter,84,5,ff,26,482749,5747,0.0,3.08,3.16,0.12
172,house-grey-box_blur-10iter,84,5,ff,28,434832,5176,0.0,3.42,3.51,0.12
173,house-grey-box_blur-10iter,84,5,ff,30,400882,4772,0.0,3.71,3.80,0.12


In [110]:
mean = df.groupby(["type", "threads"]).mean().reset_index().sort_values(by=["type", "threads"])
std = df.groupby(["type", "threads"]).std().reset_index().sort_values(by=["type", "threads"])

In [112]:
# # insert new row in position 0 with the following values
# seq = mean[mean['type'] == 'seq']


# for core in pd.unique(mean["cores"]).tolist():
#     # insert row in position 0 without substitution
#     mean.append(["ideal", core, 0, seq["completion time"] / core, core, core, core, core])

In [113]:
# split into dictionary
means = {name:d for name, d in mean.groupby(["type"])}
std = {name:d for name, d in std.groupby(["type"])}
types = means.keys()
threads = pd.unique(mean["threads"]).tolist()

In [124]:

fig = go.Figure()
for type in types:
    fig.add_trace(go.Scatter(x=threads, y=means[type]["completion time"], name=type, error_y=dict(type="data", array=std[type]["completion time"])))

# Check if this is ideal
# fig.add_trace(go.Scatter(x=threads, y=means["threads"]["completion time"], name="ideal", line=dict(color="black")))


fig.update_xaxes(type="log")
fig.update_layout(
    title="Completion time (us) vs. Number of Threads (log scale)",
    xaxis_title="Number of Threads",
    yaxis_title="Completion time",
    xaxis = dict(
        tickmode = 'array',
        tickvals = threads,
    ),
)
fig.show()

In [127]:

fig = go.Figure()
for type in types:
    fig.add_trace(go.Scatter(x=threads, y=means[type]["service time"], name=type, error_y=dict(type="data", array=std[type]["service time"])))

# Check if this is ideal
# fig.add_trace(go.Scatter(x=means["threads"]["cores"], y=means["threads"]["cores"], name="ideal", line=dict(color="black")))

fig.update_xaxes(type="log")
fig.update_layout(
    title="Service time (us) vs. Number of Threads (log scale)",
    xaxis_title="Number of Threads",
    yaxis_title="Service time",
    xaxis = dict(
        tickmode = 'array',
        tickvals = threads,
    )
)
fig.show()

In [130]:

types = means.keys()
fig = go.Figure()
for type in types:
    fig.add_trace(go.Scatter(x=threads, y=means[type]["speedup"], name=type, error_y=dict(type="data", array=std[type]["speedup"])))

fig.add_trace(go.Scatter(x=threads, y=threads, name="ideal", line=dict(color="black")))

# fig.update_xaxes(type="log")
# fig.update_yaxes(type="log")
fig.update_layout(
    title="Speedup vs. Number of Threads (log scale)",
    xaxis_title="Number of Threads",
    yaxis_title="Speedup",
    xaxis = dict(
        tickmode = 'array',
        tickvals = threads,
    ),
)
fig.show()

In [131]:

fig = go.Figure()
for type in types:
    fig.add_trace(go.Scatter(x=threads, y=means[type]["scalability"], name=type, error_y=dict(type="data", array=std[type]["scalability"])))

# Check if this is ideal
fig.add_trace(go.Scatter(x=threads, y=threads, name="ideal", line=dict(color="black")))

fig.update_xaxes(type="log")
fig.update_yaxes(type="log")
fig.update_layout(
    title="Scalability vs. Number of Threads (log scale)",
    xaxis_title="Number of Threads",
    yaxis_title="Scalability",
    xaxis = dict(
        tickmode = 'array',
        tickvals = threads,
    )
)
fig.show()

In [136]:

fig = go.Figure()
for type in types:
    fig.add_trace(go.Scatter(x=threads, y=means[type]["efficiency"], name=type, error_y=dict(type="data", array=std[type]["efficiency"])))

# Check if this is ideal
# array with number of ones equals to 4
ones = np.ones(4)
fig.add_trace(go.Scatter(x=threads, y=np.ones(len(threads)), name="ideal", line=dict(color="black")))

# fig.update_xaxes(type="log")
# fig.update_yaxes(type="log")
fig.update_layout(
    title="Efficiency vs. Number of Threads",
    xaxis_title="Number of Threads (log scale)",
    yaxis_title="Efficiency",
    xaxis = dict(
        tickmode = 'array',
        tickvals = threads,
    ),
)
fig.show()